In [9]:
import xgboost as xgb
import shap
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error as rmse
from shap_analysis import SHAPAnalyzer, function_map
import optuna

In [2]:
df = pd.read_csv("thesis_cleaned_transformed_histone_dataset_categorical(H3K23me1).tsv", sep="\t")

X = df.drop(columns = ['Gene Expression (FPKM)_log'])
y = df['Gene Expression (FPKM)_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
def evaluate_model(X_train, X_test, y_train, y_test, params=None, optuna_study=None):
    """
    Evaluate the model either using the provided parameters or the Optuna study's best trial.
    
    Parameters:
    - X_train (pd.DataFrame): Training feature data.
    - X_test (pd.DataFrame): Test feature data.
    - y_train (pd.Series): Training target data.
    - y_test (pd.Series): Test target data.
    - params (dict): Parameters for the XGBoost model. If None, use `optuna_study`.
    - optuna_study (optuna.Study): Optuna study object containing the best trial.
    
    Returns:
    - model: Trained XGBoost model.
    - total_score (float): The evaluation score based on SHAP analysis.
    - result_summary (dict): Summary of matches and mismatches.
    - results_df (pd.DataFrame): Detailed SHAP results for features.
    """
    # Determine parameters: Use the best trial from Optuna if no parameters are provided
    if optuna_study is not None and params is None:
        params = optuna_study.best_trial.params
        params.update({
            'booster': 'gbtree',
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'device': 'cuda'
        })

    # Prepare the data
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Train the model
    model = xgb.train(
        params, 
        dtrain, 
        num_boost_round=100, 
        evals=[(dtest, 'validation')],
        early_stopping_rounds=10, 
        verbose_eval=False
    )

    # SHAP analysis
    background_indices = np.random.choice(X_train.shape[0], size=200, replace=False)
    background_sample = X_train.iloc[background_indices]

    sample_indices = np.random.choice(X_test.shape[0], size=200, replace=False)
    sample = X_test.iloc[sample_indices]

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)

    # Use the SHAPAnalyzer to calculate metrics
    analyzer = SHAPAnalyzer(X_test, shap_values, function_map)
    analyzer.calculate_high_value_shap_means()

    results_df, total_score, result_summary = analyzer.get_results()

    print(f"Results Summary: {result_summary}")

    return model, total_score, result_summary, results_df


In [4]:
def feature_selection(X_train, X_test, y_train, y_test):
    remaining_features = X_train.columns.tolist()
    selected_features = []

    while remaining_features:
        print(f"Remaining Features: {remaining_features}")
        reduced_X_train = X_train[remaining_features]
        reduced_X_test = X_test[remaining_features]

        model, total_score, result_summary, results_df = evaluate_model(
            reduced_X_train, reduced_X_test, y_train, y_test
            )

        mismatch_features = results_df[results_df["Result"] == "Mismatch"]
        
        if mismatch_features.empty:
            selected_features = remaining_features
            break

        mismatch_to_remove = mismatch_features.sort_values(
            by = "Mean SHAP Value (High)", ascending = False
        ).iloc[0]['Features']

        print(f"Removing mismatch feature: {mismatch_to_remove}")

        remaining_features.remove(mismatch_to_remove)

    return selected_features

In [6]:
def feature_selection_with_optuna(X_train, X_test, y_train, y_test, function_map):
    remaining_features = X_train.columns.tolist()
    selected_features = []

    def optuna_objective(trial, X_train, X_test, y_train, y_test):
        params = {
            'booster': 'gbtree',
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'device': 'cuda',
            'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.05),
            'max_depth': trial.suggest_int("max_depth", 10, 30),
            'subsample': trial.suggest_float("subsample", 0.4, 1.0),
            'gamma': trial.suggest_float("gamma", 0.0, 1.0),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
            'lambda': trial.suggest_float("lambda", 1e-3, 15),
            'alpha': trial.suggest_float("alpha", 1e-3, 15),
    }

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_test, label=y_test)

        model = xgb.train(params, dtrain, num_boost_round=100,
                          evals=[(dvalid, 'validation')],
                          early_stopping_rounds=10,
                          verbose_eval=False)

        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)

        analyzer = SHAPAnalyzer(X_test, shap_values, function_map)
        analyzer.calculate_high_value_shap_means()

        results_df, total_score, result_summary = analyzer.get_results()
        total_mismatches = result_summary.get("Mismatch", 0)
        total_matches = result_summary.get("Match", 0)
        accuracy = total_matches / (total_matches + total_mismatches) if (total_matches + total_mismatches) > 0 else 0

        preds = model.predict(dvalid)
        rmse_score = rmse(y_test, preds)

        # Define the objective score
        alpha = 1.0  # Weight for accuracy
        beta = 0.1   # Weight for rmse_score to penalize
        objective_score = alpha * accuracy - beta * rmse_score
        
        trial.set_user_attr("result_summary", result_summary)
        trial.set_user_attr("results_df", results_df)
        
        return objective_score

    while remaining_features:
        print(f"Remaining Features: {remaining_features}")
        
        # Run Optuna for the current subset of features
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: optuna_objective(trial, 
                                                      X_train[remaining_features], 
                                                      X_test[remaining_features], 
                                                      y_train, y_test), 
                       n_trials=40)
        
        best_params = study.best_params
        print(f"Best Parameters: {best_params}")

        # Evaluate model with the current subset
        reduced_X_train = X_train[remaining_features]
        reduced_X_test = X_test[remaining_features]

        model, total_score, result_summary, results_df = evaluate_model(
            reduced_X_train, reduced_X_test, y_train, y_test, optuna_study=study
)

        mismatch_features = results_df[results_df["Result"] == "Mismatch"]

        if mismatch_features.empty:
            selected_features = remaining_features
            break

        # Sort mismatching features and remove the highest impact one
        mismatch_to_remove = mismatch_features.sort_values(
            by="Mean SHAP Value (High)", ascending=False
        ).iloc[0]["Features"]

        print(f"Removing mismatch feature: {mismatch_to_remove}")
        remaining_features.remove(mismatch_to_remove)

    return selected_features, study


In [11]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

import warnings
warnings.filterwarnings("ignore", message="Saving into deprecated binary model format")


total_matches = 0
total_mismatches = 0

features = function_map["Features"]
known_functions = function_map["Known Function"]

feature_scores = {}

for i, feature_to_leave_out in enumerate(features):
    print(f"\n--- Performing LOO for feature {feature_to_leave_out} ---\n")

    reduced_X_train = X_train.drop(columns = [feature_to_leave_out])
    reduced_X_test = X_test.drop(columns = [feature_to_leave_out])

    selected_features, study = feature_selection_with_optuna(reduced_X_train, reduced_X_test, y_train, y_test, function_map)

    final_X_train = X_train[selected_features + [feature_to_leave_out]]
    final_X_test = X_test[selected_features + [feature_to_leave_out]]

    print(f"\n--- Evaluating model with feature {feature_to_leave_out} added back ---\n")

    model, total_score, result_summary, results_df = evaluate_model(
        final_X_train, final_X_test, y_train, y_test, optuna_study=study
        )

    known_function = known_functions[i]
    feature_result = results_df[results_df["Features"] == feature_to_leave_out]

    if not feature_result.empty:
        match_result = feature_result["Result"].values[0]
        print(f"Feature: {feature_to_leave_out}, Known Function: {known_function}, Result: {match_result}")

        if match_result == "Match":
            total_matches += 1
        else:
            total_mismatches += 1

        feature_scores[feature_to_leave_out] = {
            "Total Score": total_score,
            "Known Function": known_function,
            "Result": match_result
        }

total_features = total_matches + total_mismatches
accuracy = total_matches / total_features if total_features > 0 else 0

print("\n--- Feature Analysis Results ---")

for feature, score in feature_scores.items():
    print(f"Feature: {feature}, Total Score: {score['Total Score']}, Known Function: {score['Known Function']}, Result: {score['Result']}")

print("\n--- Overall Accuracy ---")
print(f"Total Matches: {total_matches}")
print(f"Total Mismatches: {total_mismatches}")
print(f"Accuracy: {accuracy:.2f}")


--- Performing LOO for feature H3K4me1 ---

Remaining Features: ['H2A.2', 'H2A.13', 'H3K4me3', 'H3K36me3', 'H4K5Ac', 'H2A.Z.11', 'expression_category', 'H3K27me3_log', 'H3K9Ac_log', 'cpg_percentage_log', 'H3K9K14Ac_log', 'H3Ac_log', 'chh_percentage_log', 'H4K20me1_log', 'chg_percentage_log', 'H2A.W.7_log', 'H3K9me1_log', 'H2A.W.6_log', 'H2A.Z.9_log', 'H3K23me1_log', 'H3K9me2_log']


[03:31:34] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[03:31:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[03:31:40] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[03:31:47] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using 

Best Parameters: {'learning_rate': 0.012236880496338871, 'max_depth': 28, 'subsample': 0.4745770184811099, 'gamma': 0.29292571020236186, 'colsample_bytree': 0.9377761977062267, 'lambda': 6.4842569520924815, 'alpha': 13.953882512056131}


[03:34:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Results Summary: {'Match': 14, 'Mismatch': 1, 'No Evaluation': 1}
Removing mismatch feature: H4K5Ac
Remaining Features: ['H2A.2', 'H2A.13', 'H3K4me3', 'H3K36me3', 'H2A.Z.11', 'expression_category', 'H3K27me3_log', 'H3K9Ac_log', 'cpg_percentage_log', 'H3K9K14Ac_log', 'H3Ac_log', 'chh_percentage_log', 'H4K20me1_log', 'chg_percentage_log', 'H2A.W.7_log', 'H3K9me1_log', 'H2A.W.6_log', 'H2A.Z.9_log', 'H3K23me1_log', 'H3K9me2_log']


...

Best Parameters: {'learning_rate': 0.018064971807926218, 'max_depth': 20, 'subsample': 0.5534823057473445, 'gamma': 0.9191159761298259, 'colsample_bytree': 0.6750563081482891, 'lambda': 6.403741492196459, 'alpha': 8.107782194086186}


[17:43:53] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Results Summary: {'Match': 14}

--- Evaluating model with feature H3K23me1_log added back ---



[17:43:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Results Summary: {'Match': 13, 'Mismatch': 1, 'No Evaluation': 1}
Feature: H3K23me1_log, Known Function: Unknown, Result: No Evaluation

--- Feature Analysis Results ---
Feature: H3K4me1, Total Score: 1.3042715121991932, Known Function: Activating, Result: Mismatch
Feature: H3K9me2_log, Total Score: 1.8885597929474898, Known Function: Repressive, Result: Match
Feature: H3K4me3, Total Score: 1.8975745285861194, Known Function: Activating, Result: Match
Feature: H3K36me3, Total Score: 1.6590399200795218, Known Function: Activating, Result: Match
Feature: H4K5Ac, Total Score: 1.583871609531343, Known Function: Activating, Result: Mismatch
Feature: H3K27me3_log, Total Score: 1.6270086739677936, Known Function: Repressive, Result: Match
Feature: H3K9Ac_log, Total Score: 1.4494410338811576, Known Function: Activating, Result: Match
Feature: cpg_percentage_log, Total Score: 1.7572835902683437, Known Function: Repressive, Result: Match
Feature: chh_percentage_log, Total Score: 1.74529638929379